In [2]:
import requests
import mwparserfromhell
import pandas as pd

In [77]:
def getGameplay(game):
    try:
        # Get page based on serch
        session = requests.Session()

        url = "https://en.wikipedia.org/w/api.php"

        search = game + " video game"

        params = {
            "action": "query",
            "format": "json",
            "list": "search",
            "srsearch": search
        }

        request = session.get(url=url, params=params)
        data = request.json()

        page = data['query']['search'][0]['title']

        # Get gameplay section
        params = {
            "action": "parse",
            "format": "json",
            "page": page,
            "prop": "sections"
        }

        request = session.get(url=url, params=params)
        data = request.json()

        index = False

        for section in data['parse']['sections']:
            if section['line'] == 'Gameplay':
                index = section['index']
                break

        if index==False:
            return False

        # Get section wikitext
        params = {
            "action": "parse",
            "format": "json",
            "page": page,
            "section": index,
            "prop": "wikitext"
        }

        request = session.get(url=url, params=params)
        data = request.json()


        wikicode = data['parse']['wikitext']['*']

        # Parse wikicode
        parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()

        return [page, parsed_wikicode]
    except:
        return [False, False]
        print("Error in getGameplay")

In [78]:
%%time
for i in range(45):
    page = i
    games_df = pd.read_json("https://steamspy.com/api.php?request=all&page=" + str(page)).transpose()
    games_df['gameplay'] = games_df['name'].apply((lambda x: getGameplay(x)))
    games_df.to_pickle('./gamedata/page_' + str(page) + '.pkl')
    print('page_' + str(page) + '.pkl saved.')

page_0.pkl saved.
page_1.pkl saved.
page_2.pkl saved.
page_3.pkl saved.
page_4.pkl saved.
page_5.pkl saved.
page_6.pkl saved.
page_7.pkl saved.
page_8.pkl saved.
page_9.pkl saved.
page_10.pkl saved.
page_11.pkl saved.
page_12.pkl saved.
page_13.pkl saved.
page_14.pkl saved.
page_15.pkl saved.
page_16.pkl saved.
page_17.pkl saved.
page_18.pkl saved.
page_19.pkl saved.
page_20.pkl saved.
page_21.pkl saved.
page_22.pkl saved.
page_23.pkl saved.
page_24.pkl saved.
page_25.pkl saved.
page_26.pkl saved.
page_27.pkl saved.
page_28.pkl saved.
page_29.pkl saved.
page_30.pkl saved.
page_31.pkl saved.
page_32.pkl saved.
page_33.pkl saved.
page_34.pkl saved.
page_35.pkl saved.
page_36.pkl saved.
page_37.pkl saved.
page_38.pkl saved.
page_39.pkl saved.
page_40.pkl saved.
page_41.pkl saved.
page_42.pkl saved.
page_43.pkl saved.
page_44.pkl saved.
Wall time: 6h 27min 10s


In [3]:
gamedata_df = pd.DataFrame()

for i in range(45):
    df = pd.read_pickle("./gamedata/page_" + str(i) + ".pkl")
    gamedata_df = gamedata_df.append(df)
    
sorted_df = gamedata_df.sort_values(by=['positive'], ascending=False)
sorted_df = sorted_df[sorted_df['gameplay']!= False]
sorted_df = sorted_df[sorted_df['positive'] > 10]
sorted_df.shape

(15111, 18)

In [4]:
sorted_df['gameplay'].sample(25)

284410                                        [False, False]
570150                                        [False, False]
1256920    [Doom (1993 video game), Gameplay\n\nthumb|rig...
1470320                                       [False, False]
1215200                                       [False, False]
402850     [Lemmings (video game), Gameplay\nthumb|left|L...
680420     [Outriders (video game), Gameplay\nOutriders i...
490300     [Overgrowth (video game),  Gameplay \nOvergrow...
341940     [Hatred (video game),  Gameplay \nleft|thumb|"...
91810      [Ancients of Ooga, Gameplay\nAncient of Ooga i...
925440                                        [False, False]
410710     [System Shock,  Gameplay \nthumb|left|The play...
550990     [Viktor, a Steampunk Adventure, Gameplay\nThe ...
599460     [Final Fantasy (video game), Gameplay\nFinal F...
555180                                        [False, False]
111800     [Blocks That Matter, Gameplay\nthumb|Gameplay ...
1305420                 